<a href="https://colab.research.google.com/github/gdoteof/neuralnet_stuff/blob/master/kaggle_whales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install fast.ai dependencies.  

Here you want to make sure you have the gpu runtime selected.

![alt text](https://i.imgur.com/tCvtjwC.png)

The below will download and run a script to install fast.ai dependencies.

In [0]:
  !curl https://course-v3.fast.ai/setup/colab | bash

###Check GPU

The following should output:


```
1.0.0
True
True```



In [0]:

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.backends.cudnn.enabled)

1.0.0
True
True


###(Optional) Connect Google Drive

Go the the URL, give permissions on your google account, and copy the code in the box that is presented when you run the following code.

In [69]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


###Download data from Kaggle

Generate an API Token from Kaggle.  They will give you a file called kaggle.json.  Inside that file there are two values you need to fill in for `KAGGLE_USERNAME` and `KAGGLE_KEY`

In [92]:
!KAGGLE_USERNAME=gdoteof KAGGLE_KEY=1xxxxxxxxxxxxxxxxxxxxxxxxxx2 kaggle competitions download -c humpback-whale-identification


sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
  0% 0.00/594k [00:00<?, ?B/s]
100% 594k/594k [00:00<00:00, 71.1MB/s]
100% 1.34G/1.35G [00:17<00:00, 55.8MB/s]
100% 1.35G/1.35G [00:17<00:00, 81.3MB/s]
100% 4.15G/4.16G [01:11<00:00, 51.1MB/s]
100% 4.16G/4.16G [01:11<00:00, 62.0MB/s]


 {BASE}		       data			        updatestatus
 clouderizer	       drive			        updatestatus.1
 clouderizer_init.sh   install_clouderizer_service.sh   updatestatus.2
 clouderizer.jar       kaggle.json		        updatestatus.3
 clouderizer.service   models			        updatestatus.4
 colab_init.sh	      'My Drive'		        updatestatus.5
 colab.py	       __pycache__
 course-v3	       sample_submission.csv


Alright, now let's move it all to our google drive.  

In [0]:
BASE = "/content/drive/My Drive/ai/whales"

In [0]:
!mkdir -p "{BASE}/train"

In [0]:
!mv test.zip train.zip train.csv "{BASE}"

In [45]:
!echo {BASE}

/content/My Drive/ai/whales


In [93]:
!unzip "{BASE}"/train.zip -d "{BASE}"

unzip:  cannot find or open /content/drive/My Drive/ai/whales/train.zip, /content/drive/My Drive/ai/whales/train.zip.zip or /content/drive/My Drive/ai/whales/train.zip.ZIP.


In [0]:
!mv "{BASE}"/*.jpg "{BASE}"/train

In [0]:
!mkdir -p "{BASE}/test"

In [0]:
!unzip "{BASE}"/test.zip -d "{BASE}/test"

##Build our databunch!

First we set up some meta-parameters (size of the input image to the net and the batch size)

In [0]:
sz=128
bs=8

In [0]:
tfms = get_transforms(do_flip=True)
data = ImageDataBunch.from_csv('.', folder=f'{BASE}/train', csv_labels="{BASE}/train.csv", ds_tfms=tfms, sz=sz, bs=bs)

In [58]:
!ls "/content/My Drive/ai"

whales
